In [1]:
from src.houses import *
from src.agents import *
from src.bidding import *
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker # to remove 1e6 base from the x axis on plots

In [2]:
n = 1_000_000
agents = generate_agents(n)
size = sys.getsizeof(agents)
print(f"The agents array takes up {size} bytes of memory, which is {size/(1024**3):.6f} GB of memory")
print(agents)

The agents array takes up 31000112 bytes of memory, which is 0.028871 GB of memory
[(     0,  69890.98095585, 1, -1, False, -1, 0., 0.658537 )
 (     1, 472100.95258001, 7, -1, False, -1, 0., 0.72064  )
 (     2, 874942.79675377, 8, -1, False, -1, 0., 0.6024342) ...
 (999997, 229074.13927238, 5, -1, False, -1, 0., 0.6468734)
 (999998, 489773.70284395, 7, -1, False, -1, 0., 0.7484641)
 (999999, 919537.4270622 , 8, -1, False, -1, 0., 0.6436214)]


In [3]:
freq, total = get_freq_and_total(agents)
proportions = get_proportion(freq, total)
agents = check_happiness(agents, proportions)
print(agents)

[(     0,  69890.98095585, 1, -1, False, -1, 0., 0.658537 )
 (     1, 472100.95258001, 7, -1, False, -1, 0., 0.72064  )
 (     2, 874942.79675377, 8, -1, False, -1, 0., 0.6024342) ...
 (999997, 229074.13927238, 5, -1, False, -1, 0., 0.6468734)
 (999998, 489773.70284395, 7, -1, False, -1, 0., 0.7484641)
 (999999, 919537.4270622 , 8, -1, False, -1, 0., 0.6436214)]


In [4]:
utility = get_utilities(agents, proportions)
print(utility)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
bids, neighborhoods_chosen = place_bid(agents, utility)
print(bids)

[ 20967.29428675 141630.285774   262482.83902613 ...  68722.24178171
 146932.11085319 275861.22811866]


In [6]:
houses = initialize_houses(agents)
print(houses)

[(     0, -1,  1, 100000.) (     1, -1, 18, 100000.)
 (     2, -1, 63, 100000.) ... (999997, -1, 95, 100000.)
 (999998, -1, 22, 100000.) (999999, -1, 89, 100000.)]


In [7]:
agent_house_mapping(agents, houses)
priced_out_mask = check_priced_out(agents, houses, proportions)
evict_priced_out(agents, houses, priced_out_mask)

In [8]:
agents, houses, cutoff_bids = allocate_houses(agents, houses, bids, neighborhoods_chosen)
print(f"Iteration 0: {round(np.mean(agents["happy"])*100,4)}% of agents are happy")
print(f"{round(np.sum(agents["neighborhood"]==-1)/n,4)}% of agents are homeless")
# happiness check -> bids -> price update -> assignment -> happiness check
max_iter = 100
count = 1
while not np.all(agents["happy"]):
    freq, total = get_freq_and_total(agents)
    proportions = get_proportion(freq, total)

    priced_out_mask = check_priced_out(agents, houses, proportions)
    evict_priced_out(agents, houses, priced_out_mask)

    utilities = get_utilities(agents, proportions)
    bids, neighborhoods_chosen = place_bid(agents, utilities)
    agents, houses, cutoff_bids = allocate_houses(agents, houses, bids, neighborhoods_chosen)
    houses = update_prices(houses, bids, neighborhoods_chosen, cutoff_bids)
    
    freq, total = get_freq_and_total(agents)
    proportions = get_proportion(freq, total)
    agents = check_happiness(agents, proportions)
    print(f"Iteration {count}: {round(np.mean(agents["happy"])*100,4)}% of agents are happy")
    print(f"{round(np.sum(agents["neighborhood"]==-1)*100/n,4)}% of agents are homeless")
    print()
    count += 1
    if count >= max_iter:
        break

# runtime:
# 1 mil agents, not plugged in: 3m 20.9s
# 1 mil agents, plugged in: 2m 40.6s
# 30 mil agents (approx delhi popln): failed to run - out of memory

Iteration 0: 0.0% of agents are happy
0.0058% of agents are homeless
Iteration 1: 55.5922% of agents are happy
0.5766% of agents are homeless

Iteration 2: 38.305% of agents are happy
35.5389% of agents are homeless

Iteration 3: 34.2728% of agents are happy
45.1907% of agents are homeless

Iteration 4: 33.0721% of agents are happy
46.6326% of agents are homeless

Iteration 5: 32.671% of agents are happy
46.5552% of agents are homeless

Iteration 6: 33.053% of agents are happy
46.4893% of agents are homeless

Iteration 7: 33.3374% of agents are happy
46.2748% of agents are homeless

Iteration 8: 33.6671% of agents are happy
46.0185% of agents are homeless

Iteration 9: 33.8878% of agents are happy
45.8772% of agents are homeless

Iteration 10: 34.0027% of agents are happy
45.6232% of agents are homeless

Iteration 11: 34.0261% of agents are happy
45.544% of agents are homeless

Iteration 12: 34.2293% of agents are happy
45.38% of agents are homeless

Iteration 13: 34.3035% of agents ar

In [9]:
for i in range(-1, np.max(agents["neighborhood"]+1)):
    print(f"Neighborhood {i}: {np.sum(agents["neighborhood"]==i)} agents")

Neighborhood -1: 70150 agents
Neighborhood 0: 9068 agents
Neighborhood 1: 9147 agents
Neighborhood 2: 9043 agents
Neighborhood 3: 9153 agents
Neighborhood 4: 9154 agents
Neighborhood 5: 8931 agents
Neighborhood 6: 9229 agents
Neighborhood 7: 9228 agents
Neighborhood 8: 9275 agents
Neighborhood 9: 9158 agents
Neighborhood 10: 9047 agents
Neighborhood 11: 9217 agents
Neighborhood 12: 8860 agents
Neighborhood 13: 8985 agents
Neighborhood 14: 9218 agents
Neighborhood 15: 9158 agents
Neighborhood 16: 9139 agents
Neighborhood 17: 9009 agents
Neighborhood 18: 9133 agents
Neighborhood 19: 9105 agents
Neighborhood 20: 9115 agents
Neighborhood 21: 9148 agents
Neighborhood 22: 9066 agents
Neighborhood 23: 8973 agents
Neighborhood 24: 9083 agents
Neighborhood 25: 9355 agents
Neighborhood 26: 9031 agents
Neighborhood 27: 9174 agents
Neighborhood 28: 9034 agents
Neighborhood 29: 8915 agents
Neighborhood 30: 9054 agents
Neighborhood 31: 8984 agents
Neighborhood 32: 9167 agents
Neighborhood 33: 9171 a

In [ ]:
def run_sim(n, max_iter=100):
    # initialize agents and houses fresh each run
    agents = generate_agents(n)
    houses = initialize_houses(agents)
    
    freq, total = get_freq_and_total(agents)
    proportions = get_proportion(freq, total)

    priced_out_mask = check_priced_out(agents, houses, proportions)
    evict_priced_out(agents, houses, priced_out_mask)

    utilities = get_utilities(agents, proportions)
    bids, neighborhoods_chosen = place_bid(agents, utilities)
    agents, houses, cutoff_bids = allocate_houses(agents, houses, bids, neighborhoods_chosen)
    houses = update_prices(houses, bids, neighborhoods_chosen, cutoff_bids)

    freq, total = get_freq_and_total(agents)
    proportions = get_proportion(freq, total)
    agents = check_happiness(agents, proportions)

    count = 1
    while not np.all(agents["happy"]):
        freq, total = get_freq_and_total(agents)
        proportions = get_proportion(freq, total)

        priced_out_mask = check_priced_out(agents, houses, proportions)
        evict_priced_out(agents, houses, priced_out_mask)

        utilities = get_utilities(agents, proportions)
        bids, neighborhoods_chosen = place_bid(agents, utilities)
        agents, houses, cutoff_bids = allocate_houses(agents, houses, bids, neighborhoods_chosen)
        houses = update_prices(houses, bids, neighborhoods_chosen, cutoff_bids)
        
        freq, total = get_freq_and_total(agents)
        proportions = get_proportion(freq, total)
        agents = check_happiness(agents, proportions)

        count += 1
        if count >= max_iter:
            break
    
    # collect final stats
    happiness = np.mean(agents["happy"]) * 100
    homelessness = np.sum(agents["neighborhood"] == -1) * 100 / n
    return happiness, homelessness

# Monte Carlo loop
n = 1_000_000
runs = 30
results = [run_sim(n) for _ in range(runs)]

happiness_vals, homelessness_vals = zip(*results)

plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.hist(happiness_vals, bins=10, edgecolor="black")
plt.xlabel("Final Happiness (%)")
plt.ylabel("Frequency")
plt.title("Distribution of Happiness (30 runs)")

plt.subplot(1,2,2)
plt.hist(homelessness_vals, bins=10, edgecolor="black")
plt.xlabel("Final Homelessness (%)")
plt.ylabel("Frequency")
plt.title("Distribution of Homelessness (30 runs)")

plt.tight_layout()
plt.show()

# first run: 76m 18.7s = 4578.7s
# after skipping utility check for happy agents: 71m 16.6s = 4276.6s (~6.6% speedup)

In [ ]:
# plot the lognormal agents income distribution
incomes = agents["income"]

# Cut off at, say, the 99.99th percentile for visualization
cutoff = np.percentile(incomes, 99.0)
incomes_percentile = incomes[incomes <= cutoff]

fig, axes = plt.subplots(2,1,figsize = (12,12)) # one plot for actual income distr, one with top 1% cut off
axes[0].hist(incomes, bins = 500, density = True)
axes[0].set_title("Lognormal income distribution of agents")
axes[0].set_xlabel("Income per year in Rupees")
axes[0].set_ylabel("Density")
# format x-axis numbers with commas
axes[0].xaxis.set_major_formatter(mticker.StrMethodFormatter('{x:,.0f}'))

# with top 1% cut off
axes[1].hist(incomes_percentile, bins = 500, density = True)
axes[1].set_title("Lognormal income distribution of agents (top 1% exlcuded for a better view)")
axes[1].set_xlabel("Income per year in Rupees")
axes[1].set_ylabel("Density")
axes[1].xaxis.set_major_formatter(mticker.StrMethodFormatter('{x:,.0f}'))
plt.show()